(sec-mpi-point2point)=
# 点对点通信

最简单的通信模式是点对点（Point-to-Point）通信，点对点通信又分为阻塞式（Blocking）和非阻塞式（Non-Blocking）。实现点对点时主要考虑两个问题：

* 如何控制和识别不同的进程。比如，想让进程 0 给进程 1 发消息。
* 如何控制数据的读写和收发。多大的数据，数据类型是什么。

## 发送与接收

[`Comm.send()`](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.send) 和 [`Comm.recv()`](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.recv) 分别用来阻塞式地发送和接收数据。

`Comm.send(obj, dest, tag=0)` 的参数主要是 `obj` 和 `dest`。`obj` 就是我们想要发送的数据，数据可以是 Python 内置的数据类型，比如 `list` 和 `dict` 等，也可以是 NumPy 的 `ndarray`，甚至是 GPU 上的 CuPy 数据。{numref}`sec-mpi-hello-world` 我们介绍了 Communicator 和 Rank，可以通过 Rank 号来定位一个进程，`dest` 是 Rank 号码。`tag` 作用是标识，给程序员一个精细控制的选项，使用 `tag` 可以实现消息的有序传递和筛选。接收方可以选择只接收特定标签的消息，或者按照标签的顺序接收消息，以便更加灵活地控制消息的发送和接收过程。

## 案例1：发送 Python 对象

比如，我们发送一个 Python 对象。Python 对象在通信过程中的序列化使用的是 [pickle](https://docs.python.org/3/library/pickle.html#module-pickle)。

{numref}`code-mpi-send-py-object` 演示了如何发送一个 Python 对象。

```{code-block} python
:caption: send-py-object.py
:name: code-mpi-send-py-object

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1)
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    data = comm.recv(source=0)
    print(f"Received: {data}, to rank: {rank}.")
```

将这份代码保存文件为 `send-py-object.py`，在命令行中这样启动：

In [5]:
!mpiexec -np 2 python send-py-object.py

Sended: {'a': 7, 'b': 3.14}, from rank: 0.
Received: {'a': 7, 'b': 3.14}, to rank: 1.


## 案例2：发送 NumPy `ndarray`

或者发送一个 NumPy `ndarray`：

{numref}`code-mpi-send-np` 演示了如何发送一个 NumPy `ndarray`。

```{code-block} python
:caption: send-np.py
:name: code-mpi-send-np

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# 明确告知 MPI 数据类型为 int
# dtype='i', i 为 INT 的缩写
if rank == 0:
    data = np.arange(10, dtype='i')
    comm.Send([data, MPI.INT], dest=1)
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    data = np.empty(10, dtype='i')
    comm.Recv([data, MPI.INT], source=0)
    print(f"Received: {data}, to rank: {rank}.")

# MPI 自动发现数据类型
if rank == 0:
    data = np.arange(10, dtype=np.float64)
    comm.Send(data, dest=1)
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    data = np.empty(10, dtype=np.float64)
    comm.Recv(data, source=0)
    print(f"Received: {data}, to rank: {rank}.")
```
以上代码保存为 `send-np.py` 文件。

In [1]:
!mpiexec -np 2 python send-np.py

Sended: [0 1 2 3 4 5 6 7 8 9], from rank: 0.
Received: [0 1 2 3 4 5 6 7 8 9], to rank: 1.
Received: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.], to rank: 1.
Sended: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.], from rank: 0.


```{note}
这里的 `Send` 和 `Recv` 函数的首字母都大写了，因为大写的 `Send` 和 `Recv` 等方法是基于缓存（Buffer）的。对于这些基于缓存的函数，应该明确数据的类型，比如传入这样的二元组 `[data, MPI.DOUBLE]` 或三元组 `[data, count, MPI.DOUBLE]`。刚才例子中，`comm.Send(data, dest=1)` 没有明确告知 MPI 其数据类型和数据大小，是因为 MPI 对 NumPy 和 CuPy `ndarray` 做了类型的自动探测。
```

## 案例3：Master-Worker

现在我们做一个 Master-Worker 案例，共有 `size` 个进程，前 `size-1` 个进程作为 Worker，随机生成数据，最后一个进程（Rank 为 `size-1`）作为 Master，接收数据，并将数据的大小打印出来。

{numref}`code-mpi-master-worker` 对 Master 与 Worker 进程间数据的发送和接收过程进行了演示。

```{code-block} python
:caption: master-worker.py
:name: code-mpi-master-worker

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank < size - 1:
    # Worker 进程
    np.random.seed(rank)
    # 随机生成
    data_count = np.random.randint(100)
    data = np.random.randint(100, size=data_count)
    comm.send(data, dest=size - 1)
    print(f"Worker: worker ID: {rank}; count: {len(data)}")
else:
    # Master 进程
    for i in range(size - 1):
        status = MPI.Status()
        data = comm.recv(source=MPI.ANY_SOURCE, status=status)
        print(f"Master: worker ID: {status.Get_source()}; count: {len(data)}")

comm.Barrier()
```

在这个例子中，`rank` 小于 `size - 1` 的进程是 Worker，Worker 随机生成数据，并发送给最后一个进程（进程 Rank 号为 `size - 1`）。最后一个进程接收数据，并打印出接收数据的大小。

In [2]:
!mpiexec -np 8 python master-worker.py

Worker: worker ID: 0; count: 44
Worker: worker ID: 2; count: 40
Worker: worker ID: 4; count: 46
Worker: worker ID: 3; count: 24
Master: worker ID: 2; count: 40
Master: worker ID: 3; count: 24
Master: worker ID: 4; count: 46
Master: worker ID: 0; count: 44
Worker: worker ID: 5; count: 99Master: worker ID: 5; count: 99

Worker: worker ID: 1; count: 37
Master: worker ID: 1; count: 37
Worker: worker ID: 6; count: 10
Master: worker ID: 6; count: 10


## 案例 4：长方形模拟求 $\pi$ 值

对半径为 R 的圆，我们可以采用微分方法将圆切分成 N 个小长方形，当长方形数量达到无穷大的 N 时, 所有长方形总面积接近于 1/4 圆面积，如 {numref}`fig-rectangle-pi` 所示。

```{figure} ../img/ch-mpi/rectangle-pi.svg
---
width: 600px
name: fig-rectangle-pi
---
使用 N 个小长方形模拟 1/4 圆
```

假设此时有 `size` 个进程参与计算，首先求每个进程需要处理的长方形数量 (`N/size`)。每个进程各自计算长方形面积之和，并发送给 Master 进程。第一个进程作为 Master，接收各 Worker 发送数据，汇总所有矩形面积，从而近似计算出 $\pi$ 值。

{numref}`code-mpi-rectangle-pi` 演示了长方形模拟求 $\pi$ 值的过程。

```{code-block} python
:caption: rectangle-pi.py
:name: code-mpi-rectangle-pi

import math
import time

from mpi4py import MPI

communicator = MPI.COMM_WORLD
rank = communicator.Get_rank()  # 进程唯一的标识Rank
process_nums = communicator.Get_size()
"""
参数设置：
R=1
N=64*1024*1024
"""
t0 = time.time()
rect_num = 64 * 1024 * 1024
rect_width = 1 / rect_num
step_size = rect_num // process_nums

def cal_rect_area(process_no, step_size, rect_width):
    total_area = 0.0
    rect_start = (process_no * step_size + 1) * rect_width

    for i in range(step_size):
        x = rect_start + i * rect_width
        # (x,y) 对应于第i个小矩形唯一在圆弧上的顶点
        # x^2+y^2=1 => y=sqrt(1-x^2)
        rect_length = math.pow(1 - x * x, 0.5)
        total_area += rect_width * rect_length
    return total_area

# 在每个进程上执行计算
total_area = cal_rect_area(rank, step_size, rect_width)

if rank == 0:
    # Master
    for i in range(1, process_nums):
        total_area += communicator.recv(source=i)
    p_i = total_area * 4
    t1 = time.time()
    print("Simulated PI: {:.10f}, Relative Error：{:.10f}".format(p_i, abs(1 - p_i / math.pi)))
    print("Time：{:.3f}s".format(t1 - t0))
else:
    # Worker
    communicator.send(total_area, dest=0)
```

上述例子中，我们设置参数为：`R=1`, `N=64*1024*1024`，保存文件为 `rectangle_pi.py`：

In [7]:
!mpiexec -np 8 python rectangle_pi.py

Simulated PI: 3.1415926238, Relative Error：0.0000000095
Time：7.361s


## 阻塞 v.s. 非阻塞

### 阻塞

我们先分析一下阻塞式通信。`Send` 和 `Recv` 这两个基于缓存的方法：

* `Send` 直到缓存是空的时候，也就是说缓存中的数据都被发送出去后，才返回（`return`）。缓存区域可以被接下来其他的 `Send` 循环再利用。
* `Recv` 直到缓存区域填满，才返回（`return`）。

如 {numref}`mpi-communications` 所示，阻塞通信是数据完成传输，才会返回（`return`），否则一直在等待。

```{figure} ../img/ch-mpi/blocking.svg
---
width: 600px
name: blocking-communications
---
阻塞式通信示意图
```

阻塞式通信的代码更容易去设计，但容易出现死锁，比如类似下面的逻辑，Rank 1 进程产生了死锁，应该将 `Send` 和 `Recv` 调用顺序互换

```python
if rank == 0:
	comm.Send(..to rank 1..)
    comm.Recv(..from rank 1..)
else if (rank == 1): <- 该进程死锁
    comm.Send(..to rank 0..)       <- 应将 Send Revc 互换
    comm.Recv(..from rank 0..)
```

### 非阻塞

相比之下，非阻塞通信不会等待数据传输的完成。非阻塞通信可以重叠通信和计算来增强性能，即网络侧完成通信任务，CPU 侧完成计算任务。[`isend`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.isend) 和 [`irecv`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.irecv) 可用于非阻塞通信：

* `isend`：启动一个非阻塞发送操作并立即将控制返回给程序员，允许执行后续代码。

* `irecv`：启动一个非阻塞接收操作并立即将控制返回给程序员，允许执行后续代码。

非阻塞式通信调用后直接返回 [`Request`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Request.html#mpi4py.MPI.Request) 句柄（Handle），程序员接下来再对 `Request` 做处理，比如等待 `Request` 涉及的数据传输完毕。非阻塞式通信有大写的 I 或小写的 i 作为前缀，大写 I 前缀的基于缓存，小写 i 前缀的没有。[`isend`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.isend) 的函数参数与 [`send`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.send) 相差不大，只不过 `isend` 返回值是一个 `Request`。 `Request` 类提供了 `wait` 方法，显示地调用 `wait()` 可以等待数据传输完毕。用 `send` 写的阻塞式的代码，可以改为 `Isend` + [`Request.wait()`](https://mpi4py.readthedocs.io/en/stable/reference/mpi4py.MPI.Request.html#mpi4py.MPI.Request.wait) 以非阻塞方式实现。

{numref}`code-mpi-non-blocking` 展示了一个非阻塞式通信的例子。

```{code-block} python
:caption: non-blocking.py
:name: code-mpi-non-blocking

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    req = comm.isend(data, dest=1, tag=11)
    print(f"Sending: {data}, from rank: {rank}.")
    req.wait()
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    req = comm.irecv(source=0, tag=11)
    print(f"Receiving: to rank: {rank}.")
    data = req.wait()
    print(f"Received: {data}, to rank: {rank}.")
```

In [4]:
!mpiexec -np 8 python non-blocking.py

Receiving: to rank: 1.
Sending: {'a': 7, 'b': 3.14}, from rank: 0.
Sended: {'a': 7, 'b': 3.14}, from rank: 0.
Received: {'a': 7, 'b': 3.14}, to rank: 1.


{numref}`fig-non-blocking-communications` 展示非阻塞通信 `wait()` 加入后数据流的变化。

```{figure} ../img/ch-mpi/non-blocking.svg
---
width: 800px
name: fig-non-blocking-communications
---
非阻塞式通信示意图
```